In [21]:
import pyhepmc as hp
import numpy as np
from pyhepmc.view import savefig

DARK_PHOTON_PID = 4900022
NUM_EVENTS = 1000

input_file = "../pythia8315/examples/mainDisplacedPhoton.hepmc"
output_file = "darkDisplacedPhoton.hepmc"
# output_file = "halfDarkPromptPhoton.hepmc"
# output_file = "halfDarkDisplacedPhoton.hepmc"

In [22]:
# Temporary event access for testing
# with hp.open("../pythia8315/examples/mainDisplacedPhoton.hepmc") as f:
#     for event in f:
#         break
    
# event.set_units(hp.Units.GEV, hp.Units.LengthUnit.MM) # Probably not needed, but just for confidence

In [23]:
def get_decay_vertices(GenEvent: hp.GenEvent, pin_id: int, status_code: int) -> list[hp.GenVertex]:
    """Gets the decay vertices of particles with a given PID and status code.

    Args:
        GenEvent (hp.GenEvent): GenEvent to get vertices from
        pin_id (int): Particle ID to filter by
        status_code (int): Status code to filter by

    Returns:
        list[hp.GenVertex]: Decay vertices of the filtered particles
    """
    vertices = []
    vertices_ids = set()
    # print(f"Searching for decay vertices with PID {pin_id} and status {status_code}")
    for phi in [p for p in sorted(GenEvent.particles, key=lambda p: p.status) if (p.pid == pin_id) and (p.status == status_code)]:
        # print(f"Found particle with ID {phi.pid} and status {phi.status}: {phi}")
        if phi.end_vertex.id not in vertices_ids:
            vertices.append(phi.end_vertex)
            vertices_ids.add(phi.end_vertex.id)

    return vertices

In [24]:
def modify_vertex_particles_out(vertex: hp.GenVertex, new_pids: list[int]) -> None:
    """Modifies the outgoing particles of a vertex to have new PIDs.

    Args:
        vertex (hp.GenVertex): Vertex to modify
        new_pids (list[int]): List of new PIDs to assign to the outgoing particles
    """
    if len(vertex.particles_out) != len(new_pids):
        raise ValueError("Number of new PIDs must match number of outgoing particles")

    for i, particle in enumerate(vertex.particles_out):
        old_pid = particle.pid
        particle.pid = new_pids[i]
        # print(f"Changed particle PID from {old_pid} to {particle.pid}. Now: {particle}")
    # print(f"Modified vertex particles out: {vertex.particles_out}")

def modify_vertex_location(vertex: hp.GenVertex, new_location: hp.FourVector) -> None:
    """Modifies the location of a vertex.

    Args:
        vertex (hp.GenVertex): Vertex to modify
        new_location (hp.FourVector): New location to assign to the vertex
    """
    # old_location = vertex.position
    vertex.position = new_location
    # print(f"Changed vertex location from {old_location} to {vertex.position}. Now: {vertex}")

In [25]:
def get_modified_vertex_location(vertex: hp.GenVertex) -> hp.FourVector:
    """Gets the modified location of a vertex by scaling momentum four vector of parent phi.

    Args:
        vertex (hp.GenVertex): Vertex to get phi momentum from.

    Returns:
        hp.FourVector: Modified location of the vertex
    """
    pin = vertex.particles_in
    assert len(pin) == 1, "Vertex must have exactly one incoming particle"
    phi_momentum = pin[0].momentum

    scale_factor = np.random.exponential(1000)/phi_momentum.m()
    
    # Scale the phi momentum to get the new vertex position
    new_position = phi_momentum * scale_factor

    return new_position

In [26]:
def set_decays_dark(event: hp.GenEvent) -> list[hp.GenEvent]:
    """Modifies the decays of phis from phiphi>4photons to phiphi>3dark+photon.
    Each event is copied 4 times, with each copy having a different remaining photon
    
    Args:
        event (hp.GenEvent): Event to modify
        
    Returns:
        list[hp.GenEvent]: List of modified events
    """
    event_number = event.event_number
    photon_pid = 22
    dark_photon_pid = DARK_PHOTON_PID
    
    event_data = hp.GenEventData()
    
    event.write_data(event_data)
    event2 = hp.GenEvent()
    event2.read_data(event_data)
    event3 = hp.GenEvent()
    event3.read_data(event_data)
    event4 = hp.GenEvent()
    event4.read_data(event_data)
    
    events = [event, event2, event3, event4]
    decays = [dark_photon_pid, dark_photon_pid, dark_photon_pid, dark_photon_pid]

    for i, evt in enumerate(events):
        #FIXME change branching ratios here
        
        evt.event_number = event_number * 4 + i
        evt_decays = decays.copy()
        evt_decays[i] = photon_pid
        
        vertices = get_decay_vertices(evt, 54, 62)
        assert len(vertices) == len(evt_decays) // 2, f"Event {evt.event_number} has {len(vertices)} vertices, not expected {len(evt_decays) // 2}"
        for j, vtx in enumerate(vertices):
            modify_vertex_particles_out(vtx, evt_decays[j*2:j*2+2])
            loc = get_modified_vertex_location(vtx) # Uncomment to modify vertex location
            modify_vertex_location(vtx, loc) # Uncomment to modify vertex location

    return events

In [27]:
def set_decays_half_dark(event: hp.GenEvent) -> list[hp.GenEvent]:
    """Modifies the decays of phis from phiphi>4photons to phiphi>2dark+2photon.
    Each event is copied 6 times, with each copy having a different pair of remaining photons"""
    event_number = event.event_number
    photon_pid = 22
    dark_photon_pid = DARK_PHOTON_PID
    
    event_data = hp.GenEventData()
    
    event.write_data(event_data)
    event2 = hp.GenEvent()
    event2.read_data(event_data)
    event3 = hp.GenEvent()
    event3.read_data(event_data)
    event4 = hp.GenEvent()
    event4.read_data(event_data)
    event5 = hp.GenEvent()
    event5.read_data(event_data)
    event6 = hp.GenEvent()
    event6.read_data(event_data)
    
    events = [event, event2, event3, event4, event5, event6]
    decays = [dark_photon_pid, dark_photon_pid, dark_photon_pid, dark_photon_pid]

    for i, evt in enumerate(events):
        #FIXME change branching ratios here
        
        evt.event_number = event_number * 6 + i
        evt_decays = decays.copy()
        if i == 0:
            evt_decays[0] = photon_pid
            evt_decays[1] = photon_pid
        elif i == 1:
            evt_decays[0] = photon_pid
            evt_decays[2] = photon_pid
        elif i == 2:
            evt_decays[0] = photon_pid
            evt_decays[3] = photon_pid
        elif i == 3:
            evt_decays[1] = photon_pid
            evt_decays[2] = photon_pid
        elif i == 4:
            evt_decays[1] = photon_pid
            evt_decays[3] = photon_pid
        elif i == 5:
            evt_decays[2] = photon_pid
            evt_decays[3] = photon_pid
        
        vertices = get_decay_vertices(evt, 54, 62)
        assert len(vertices) == len(evt_decays) // 2, f"Event {evt.event_number} has {len(vertices)} vertices, not expected {len(evt_decays) // 2}"
        for j, vtx in enumerate(vertices):
            modify_vertex_particles_out(vtx, evt_decays[j*2:j*2+2])
            loc = get_modified_vertex_location(vtx) # Uncomment to modify vertex location
            modify_vertex_location(vtx, loc)

    return events

In [28]:
def write_events_to_file(events: list[hp.GenEvent], filename: str) -> None:
    """Writes a list of hp.GenEvent objects to a .hepmc file.

    Args:
        events (list[hp.GenEvent]): List of GenEvent objects to write.
        filename (str): Path to the output .hepmc file.
    """
    with hp.open(filename, "w") as output_file:
        for event in events:
            output_file.write(event)

In [29]:
# with hp.open("../pythia8315/examples/mainDisplacedPhoton.hepmc") as f:
#     for event in f:
#         if (event.event_number % 2 == 0) and (event.event_number > 0):
#             break

# events = set_decays_half_dark(event)
# for evt in events:
#     hp.view.savefig(evt, f"event_images/event_{evt.event_number}.pdf")


In [30]:
with hp.open(input_file) as f, hp.open(output_file, "w") as output:
    for event in f:
        if event.event_number >= NUM_EVENTS: # Modify number of events here
            break
        print("Processing event:", event.event_number, end=", ")
        events = set_decays_dark(event)
        # events = set_decays_half_dark(event)
        print("Saving events:", events)
        # if (event.event_number % 2 == 0) and (event.event_number > 0):
        #     for evt in events:
        #         hp.view.savefig(evt, f"event_images/event_{evt.event_number}.pdf")
        #     break
        for evt in events:
            output.write(evt)

Processing event: 0, Saving events: [<GenEvent momentum_unit=1, length_unit=0, event_number=0, particles=909, vertices=523, run_info=GenRunInfo(tools=[], weight_names=['Weight'], attributes={})>, <GenEvent momentum_unit=1, length_unit=0, event_number=1, particles=909, vertices=523, run_info=None>, <GenEvent momentum_unit=1, length_unit=0, event_number=2, particles=909, vertices=523, run_info=None>, <GenEvent momentum_unit=1, length_unit=0, event_number=3, particles=909, vertices=523, run_info=None>]
Processing event: 1, Saving events: [<GenEvent momentum_unit=1, length_unit=0, event_number=4, particles=832, vertices=474, run_info=GenRunInfo(tools=[], weight_names=['Weight'], attributes={})>, <GenEvent momentum_unit=1, length_unit=0, event_number=5, particles=832, vertices=474, run_info=None>, <GenEvent momentum_unit=1, length_unit=0, event_number=6, particles=832, vertices=474, run_info=None>, <GenEvent momentum_unit=1, length_unit=0, event_number=7, particles=832, vertices=474, run_in